# TC detection on UM `Zoom = 7`

In [ ]:
# packages
import os
import sys
import subprocess
import numpy as np
from glob import glob
import pandas as pd
import xarray as xr
from dask.distributed import Client
from datetime import datetime, timedelta

In [ ]:
# link to self-written packages
sys.path.append("/g/data/gb02/cj0591/hk25-AusCyclones") # change to your directory
from utils.tools import write_to_filelist, clear_dir
from tempestextremes_utils.node_utils import run_stitchNodes

In [ ]:
def run_detectNodes_regional(input_filelist, detect_filelist, mpi_np=4,
                             detect_var="msl",
                             merge_dist=6.0,
                             closedcontour_commands="msl,200.0,5.5,0;_DIFF(z(300millibars),z(500millibars)),-58.8,6.5,1.0",
                             output_commands="msl,min,0;_VECMAG(u10,v10),max,2.0;zs,min,0",
                             timeinterval="6hr",
                             lonname="longitude",latname="latitude",
                             minlon=90, maxlon=180, minlat=-60, maxlat=0,
                             logdir="./log/",
                             quiet=False):

    # DetectNode command
    detectNode_command = ["mpirun", "-np", f"{int(mpi_np)}",
                            f"{os.environ['TEMPESTEXTREMESDIR']}/DetectNodes",
                            "--in_data_list",f"{input_filelist}",
                            "--out_file_list", f"{detect_filelist}",
                            "--searchbymin",f"{detect_var}",
                            "--closedcontourcmd",f"{closedcontour_commands}",
                            "--mergedist",f"{merge_dist}",
                            "--outputcmd",f"{output_commands}",
                            "--timefilter",f"{timeinterval}",
                            "--latname",f"{latname}",
                            "--lonname",f"{lonname}",
                            "--regional",
                            "--minlon",f"{minlon}",
                            "--maxlon",f"{maxlon}",
                            "--minlat",f"{minlat}",
                            "--maxlat",f"{maxlat}",
                            "--logdir",f"{logdir}",
                            ]
    
    detectNode_process = subprocess.Popen(detectNode_command,
                                          stdout=subprocess.PIPE, 
                                          stderr=subprocess.PIPE, text=True)
    
    # Wait for the process to complete and capture output
    stdout, stderr = detectNode_process.communicate()

    path,_=os.path.split(input_filelist)
    outfile=path+'/detectNodes_outlog.txt'
    with open(outfile, 'w') as file:
        file.write(stdout)
    outfile=path+'/detectNodes_errlog.txt'
    with open(outfile, 'w') as file:
        file.write(stderr)
    if not quiet:
         return stdout, stderr

TempestExtremes allows parallel running with `mpi`

In [3]:
# set dask workers
client = Client(n_workers=7)

**Set directories**  
`csv_dir` for TempestExtremes stitchNode final output  
`log_dir` for log files  
`input_dir` for input UM variables filelist  
`output_dir` for TempestExtremes detectNode output filelist  
`output_temp_dir` for temporary TempestExtremes detectNode ouput

In [ ]:
# base directory (change to your directory)
base_dir = '/g/data/gb02/cj0591/hk25-AusCyclones'

# input & output directory
csv_dir = f'{base_dir}/csv' 
log_dir = f'{base_dir}/log' # log files
input_dir = f'{base_dir}/in' # input filelist
output_dir = f'{base_dir}/out' # output filelist
output_temp_dir = f'{base_dir}/temp' # temporary for output files

# directory for TempestExtremes
os.environ['TEMPESTEXTREMESDIR']='/g/data/gb02/tempestextremes/bin'

In [8]:
# be very careful with this - it will delete everything in the directory!!!
clear_dir(log_dir)
clear_dir(input_dir)
clear_dir(output_dir)
clear_dir(output_temp_dir)

**Required variables for TC detection**  

| Variable Name                 | Level (hPa)                       |
|-------------------------------|-----------------------------------|
| Surface U-component Wind (uas)   | Surface                           |
| Surface V-component Wind (uas)   | Surface                           |
| Mean Sea Level Pressure (psl) | Surface                           |
| Geopotential height (zg)              | 500, 300                          |

**Create lists for inputfile and outputfile**

Inputfile consist of several files containing geopotential height (zg) on pressure surfaces, air pressure at mean sea level (psl), surface zonal and meridional wind speeds (uas and vas), separated by semicolons.

In [9]:
zgfiles = sorted(glob('/scratch/gb02/cj0591/um/zg/*'))
pslfiles = sorted(glob('/scratch/gb02/cj0591/um/psl/*'))
uasfiles = sorted(glob('/scratch/gb02/cj0591/um/uas/*'))
vasfiles = sorted(glob('/scratch/gb02/cj0591/um/vas/*'))

infilenames_list = []
outfilenames_list = []

for i in np.arange(0, len(zgfiles)):
    infilenames_list.append(f"{zgfiles[i]};{pslfiles[i]};{uasfiles[i]};{vasfiles[i]}")
    outfilenames_list.append(f"{output_temp_dir}/detectNode_TC_hres7_{zgfiles[i][-10:-3]}.txt")

    write_to_filelist(infilenames_list,f'{input_dir}/detectNode_TC_input_hres7.txt')
    write_to_filelist(outfilenames_list,f'{output_dir}/detectNode_TC_output_hres7.txt')

**Run TempestExtremes DetectNode**

DetectNode detects nodes  


Thresholds (`closedcontour_commands`) are applied: 

(a) `psl,200.0,5.5,0` represents that mean sea level pressure must increase by 200 Pa over a 5.5 great circle distance (GCD) from the detected node;  


(b) `_DIFF(zg(300hPa),zg(500hPa)),-6,6.5,1.0` represents that the difference between geopotential height (zg) on the 300 and 500 hPa surfaces must decrease by 6 m over a 6.5 GCD, using the maximum value of this field within 1 GCD as reference. This ensures a coherent upper-level warm core attached to the detected surface low


More details can be found in [Ullrich et al., 2021](https://gmd.copernicus.org/articles/14/5023/2021/)

In [ ]:
run_detectNodes_regional(f'{input_dir}/detectNode_TC_input_hres7.txt', # inputfile list
                         f'{output_dir}/detectNode_TC_output_hres7.txt', # outputfile list
                         7, # cores used for mpi parallel running
                         detect_var="psl", # variable used to detect nodes
                         merge_dist=6.0,   # merge distance of detected nodes are close to each other of 6.0 great circle distance (GCD)
                         closedcontour_commands="psl,200.0,5.5,0;_DIFF(zg(300hPa),zg(500hPa)),-6,6.5,1.0",
                         output_commands="psl,min,0;_VECMAG(uas,vas),max,2.0",
                         timeinterval="6hr",
                         lonname="longitude",latname="latitude",
                         minlon=90, maxlon=180, minlat=-60, maxlat=0,
                         logdir=f"{log_dir}",
                         quiet=True
                         )

We can monitor the algrithm progress through log files under `log_dir`

**Run TempestExtremes StitchNode**

StitchNode connects detected nodes in time.  


Thresholds (`threshold_condition`) are applied:  

(a) `wind,>=,10.0,10` represents that the wind magnitude must be greater than 10 m/s for at least 10 timesteps;  

(b) `lat,<=,50.0,10;lat,>=,-50.0,10` represents that the latitude for detected nodes must be within 50S and 50N for at least 10 timesteps;  


More details can be found in [Ullrich et al., 2021](https://gmd.copernicus.org/articles/14/5023/2021/)

In [8]:
# Run TempestExtremes StitchNode
run_stitchNodes(f"{output_dir}/detectNode_TC_output_hres7", # inputfile list
                f"{csv_dir}/stitchNode_TC_output_hres7.csv", # output file
                1, # cores used for mpi parallel running StitchNode run very fast with only one core
                output_filefmt="csv", # output format
                in_fmt_commands="lon,lat,msl,wind", # input format of the detectnode ouput
                range_dist=8.0, # the maximum distance (in GCD) that a node can move between two timesteps
                minim_time="54h", # the minimum lifetime of each track
                maxgap_time="24h", # the maximum duration between two timesteps
                min_endpoint_dist="", # the total distance from the strat to the end of the trajectory
                threshold_condition="wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10", # threshold
                quiet=True)